In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
#from dataPath import DATA_PATH
%matplotlib inline

import matplotlib
matplotlib.rcParams['figure.figsize'] = (4.0,4.0)
matplotlib.rcParams['image.cmap'] = 'gray'

### Set Up Trackers #################################
tracker_types = ['BOOSTING', 'MIL','KCF', 'TLD', 'MEDIANFLOW', 'GOTURN', 'CSRT', 'MOSSE']
tracker_type = tracker_types[6]

if tracker_type == 'BOOSTING':
    tracker = cv2.TrackerBoosting_create()
elif tracker_type == 'MIL':
    tracker = cv2.TrackerMIL_create()
elif tracker_type == 'KCF':
    tracker = cv2.TrackerKCF_create()
elif tracker_type == 'TLD':
    tracker = cv2.TrackerTLD_create()
elif tracker_type == 'MEDIANFLOW':
    tracker = cv2.TrackerMedianFlow_create()
elif tracker_type == 'GOTURN':
    tracker = cv2.TrackerGOTURN_create()
elif tracker_type == "CSRT":
    tracker = cv2.TrackerCSRT_create()
elif tracker_type == "MOSSE":
    tracker = cv2.TrackerMOSSE_create()
else:
    tracker = None
    print('Incorrect tracker name')
    print('Available trackers are:')
    for t in trackerTypes:
        print(t)
        
### Load the video #####################################
__location__ = os.path.realpath(os.getcwd())
filename=os.path.join(__location__, 'soccer-ball.mp4')


### Set default parameters #############################
BLUE=(255,0,0)
GREEN=(0,255,0)
RED = (0,0,255)
WhiteLower=(235,235,235)
WhiteUpper=(255,255,255)

### Set up View Window #################################
windowName = "Detect and Track Soccer Ball"
cv2.namedWindow(windowName, cv2.WINDOW_NORMAL)


### Soccer Ball Detection ##############################
def findCircle(frame):
    minRadius=45
    maxRadius=85
    prevRadius=minRadius
    finalRadius=int((minRadius+maxRadius)/2)
    
    h,w,c=frame.shape
    finalx=w  #int(w/2)
    finaly=0  #int(h/2)
    
    bIm, gIm, rIm=cv2.split(frame)
    
    blurIm=cv2.GaussianBlur(rIm, (11,11),0)
    th, binaRIm = cv2.threshold(rIm, 170, 255, cv2.THRESH_BINARY)
        
    # construct a mask for the color "green", then perform
    # a series of dilations and erosions to remove any small
    # blobs left in the mask
    kernelclose = np.ones((5,5),np.uint8)
    imgclose = cv2.morphologyEx(binaRIm, cv2.MORPH_CLOSE, kernelclose, iterations= 5)
    kernelopen = np.ones((10,10),np.uint8)
    mask=cv2.morphologyEx(imgclose, cv2.MORPH_OPEN, kernelopen, iterations=5)
    
    drawframe=frame.copy()
    
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    # Draw all the contours #last element is line width
    for cnt in contours:
    # We will use the contour moments
    # to find the centroid
        # Fit a circle
        ((x,y),radius) = cv2.minEnclosingCircle(cnt)
        
        if (radius>=minRadius and radius<=maxRadius):
            if radius> prevRadius:
                finalx=int(x)
                finaly=int(y)
                finalRadius=int(radius)
                
                prevRadius=radius
            # Mark the center
    cornerTx=int(finalx-finalRadius)
    cornerTy=int(finaly-finalRadius)
    cornerBx=int(finalx+finalRadius)
    cornerBy=int(finaly+finalRadius)
    cv2.circle(drawframe, (int(finalx),int(finaly)), int(round(finalRadius)), RED, 5)
    cv2.rectangle(drawframe, (cornerTx,cornerTy),(cornerBx,cornerBy), GREEN, 5)
            
        
    ballbox=(cornerTx, cornerTy, 2*finalRadius, 2*finalRadius)
    return ballbox

### Draw Detected Bounding Box #############################
def DrawInitialDetectedBox(frame, bbox):
    global EscCMD, gCMD
    p1 = (int(bbox[0]), int(bbox[1]))
    p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
    cv2.rectangle(frame, p1, p2, GREEN, 2, 1 )

    cv2.putText(frame,EscCMD, (20,20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.75, BLUE,2);
    cv2.putText(frame, gCMD, (20,80), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.75, GREEN,2);
    return frame

def DrawDetectedBox(frame, bbox):
    global EscCMD, gCMD
    p1 = (int(bbox[0]), int(bbox[1]))
    p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
    cv2.rectangle(frame, p1, p2, GREEN, 2, 1 )

    cv2.putText(frame,EscCMD, (20,20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.75, BLUE,2);
    cv2.putText(frame, renewDetectCMD, (20,140), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.75, GREEN,2);
    return frame

### Tracking Soccer Ball at each video frame ###############
count=0    
k=-1
EscCMD="Press Esc to terminate the video"
gCMD="Soccer ball is detected. Press g to proceed"
renewDetectCMD="Soccer ball is re-detected."

cap = cv2.VideoCapture(filename)

if (cap.isOpened()== False): 
  print("Error opening video stream or file")


while(cap.isOpened()):
  
  if k==27:
    cap.release()
    break
    
  # Capture frame-by-frame
  ret, frame = cap.read()  # Read the immediate frame
       
  if ret == True:
    
    if count==0:
        count+=1
        bbox=findCircle(frame)
        ok = tracker.init(frame, bbox)
        
        frame=DrawInitialDetectedBox(frame, bbox)
        
        # Display result
        cv2.imshow(windowName, frame)
        
        k=cv2.waitKey(0)
        
    elif k & 0xFF ==ord('g') and count==1:  #count>0
        count+=1
    elif count>1:
                
    # The update method is used to obtain the location 
    # of the new tracked object. The method returns
    # false when the track is lost. Tracking can fail 
    # because the object went outside the video frame or 
    # if the tracker failed to track the object. 
    # In both cases, a false value is returned. At this point, the detection algorithm is initiated.
    # The soccer ball is re-detected and then the tracker is re-started.
    
    # Update tracker
        ok, bbox = tracker.update(frame)
    
    # Display results.
    
        cv2.putText(frame,tracker_type + " Tracker   "+"Press Esc to close the window", (20,20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.75, BLUE,2);
        
    # Draw bounding box
        if ok:
      # Tracking success
          p1 = (int(bbox[0]), int(bbox[1]))
          p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
          cv2.rectangle(frame, p1, p2, BLUE, 2, 1)
        else :
      # Tracking failure
          cv2.putText(frame, "Tracking failure detected", (20,80), 
              cv2.FONT_HERSHEY_SIMPLEX, 0.75,RED,2)
          tracker = cv2.TrackerCSRT_create()
          bbox=findCircle(frame)
          if bbox[1]>0:
              ok = tracker.init(frame, bbox) 
              print("Re-detected at frame "+str(int(cap.get(cv2.CAP_PROP_POS_FRAMES))))
              frame=DrawDetectedBox(frame, bbox)
    # Display result
        cv2.imshow(windowName, frame)
        
        k=cv2.waitKey(25)
        
  # Renew the video
  else: 
    cap = cv2.VideoCapture(filename)

# When everything done, release the VideoCapture and VideoWriter objects
cap.release()
cv2.destroyAllWindows()
                    
### End of File ################################################


Re-detected at frame 95
Re-detected at frame 266
Re-detected at frame 500
